#검증세트

모델을 만드는 과정에서 테스트세트로 테스트를 하는 일이 많아질수록 모델은 일반화 성능이 좋은 모델이 아닌 테스트세트에 성능이 좋은 모델이 만들어진다.

즉, 테스트세트는 마지막에 한번 사용하는것이 좋다.

그렇다면 모델 테스트는 어떻게 할 것인가?

그래서 우린 검증세트도 만든다!

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine-date')

In [2]:
data = wine.drop('class', axis = 1).to_numpy()
target = wine['class'].to_numpy()

In [3]:
#훈련세트와 테스트세트를 먼저 나눈다.
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state=42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [4]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# K-fold cross validation
휸련세트를 몇부분으로 나눠서 교차로 훈련,검증세트로 사용하는 방법


In [5]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00912547, 0.00692987, 0.00727415, 0.00760841, 0.00689697]), 'score_time': array([0.00070024, 0.00065184, 0.00060058, 0.00065589, 0.00075889]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [6]:
import numpy as np
print(np.mean(scores['test_score'])) #이게 모델에서 얻을 수 있는 최상의 검증점수라고 가늠 할 수 있다.

0.855300214703487


In [7]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [8]:
#10-fold cross 하려면
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


#하이퍼파라미터 튜닝

하이퍼 파라미터 탐색은 찾아야 할 모든 변수를 전부 바꿔가면서 최적의 값을 탐색해야 한다.



In [17]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

#GridSearch에 탐색 대상 모델과 탐색 할 변수 param을 전달한다.
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
#n_jobs -> 병렬 실행에 사용할 코어 수 (-1 : 모든 코어를 사용)

In [18]:
gs.fit(train_input, train_target)
# 최적의 하이퍼 파라미터를 찾아 best_estimator_에 저장

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [19]:
dt = gs.best_estimator_
#gs의 best_estimator을 모델과 똑같이 사용 가능하다.

print(dt.score(train_input, train_target))

0.9615162593804117


In [20]:
print(gs.best_params_ )

{'min_impurity_decrease': 0.0001}


In [22]:
print(gs.cv_results_['mean_test_score'])
#각 매개변수에서 수행 한 교차검증의 평균점수

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [23]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [25]:
params = { 'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)
}
# 순서대로 노드를 분할하기 위한 불순도 감소 최소량, 트리의 깊이 제한, 노드를 나누기 위한 최소 샘플 수

In [26]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [27]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [28]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


#랜덤 서치
하이퍼 파라미터에 전달 할 매개변수의 후보값들을 미리 정하기 어려울 수 있다.

랜덤서치에는 매개변수 값의 목록(후보) 를 전달하는것이 아니라 매개변수를 샘플링 할 수 있는 확률 분포 객체를 전달한다.

In [30]:
#난수를 발생시켜주는 함수 uniform(실수) 와 radint(정수)
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([2, 8, 3, 1, 6, 8, 6, 2, 9, 8])

In [34]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([101, 104, 113, 111, 105,  93,  94,  93,  96,  90]))

In [35]:
ugen= uniform(0,1)
ugen.rvs(10)

array([0.81296331, 0.2839787 , 0.32672693, 0.33295675, 0.55981049,
       0.12124762, 0.41183714, 0.2933056 , 0.44028933, 0.2698196 ])

In [40]:
#min_samples_leaf : 리프노드가 되기 위한 최소 샘플의 개수 (어떤 노드가 분할하여 만들어 질 자식의 노드 샘플수가 이 값보다 작다면 분할하지 않음 )
params = {'min_impurity_decrease' : uniform(0.0001, 0.001), 
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25), 
          'min_samples_leaf' : randint(1, 25),
}

In [41]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f55bcabd350>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f55bcabd950>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f55bcabde90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f55bcabd610>},
                   random_state=42)

In [42]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [43]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [44]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


하이퍼 파라미터를 결정지을때는 그리드서치나 랜덤서치를 이용하는 것이 좋다.

이 경우 검증 세트를 따로 만들 필요 없이 교차검증이 이루어진다.